# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Hobbies'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [ ]:
len(sales_intermittent)

1301

In [ ]:
len(sales_lumpy)

395

In [7]:
len(sales_erratic)

99

In [ ]:
len(sales_smooth)

3594

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [8]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
73,HOBBIES_1_076_CA_1_validation,HOBBIES_1_076,HOBBIES_1,HOBBIES,CA_1,CA,0,1,2,3,...,0,2,1,0,0,0,0,1,1,0
77,HOBBIES_1_080_CA_1_validation,HOBBIES_1_080,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,12,2,0,0,7
88,HOBBIES_1_091_CA_1_validation,HOBBIES_1_091,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,7,1,3,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9292,HOBBIES_1_150_CA_4_validation,HOBBIES_1_150,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9298,HOBBIES_1_156_CA_4_validation,HOBBIES_1_156,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0
9342,HOBBIES_1_202_CA_4_validation,HOBBIES_1_202,HOBBIES_1,HOBBIES,CA_4,CA,2,1,5,3,...,0,0,0,0,0,0,0,0,0,0
9362,HOBBIES_1_223_CA_4_validation,HOBBIES_1_223,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,2,0,0,0,1,0,0,0,1,0


# User-defined functions to calculate Metrics and Croston algorithm

In [9]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [10]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [11]:
sales_pattern = sales_erratic.copy()

In [12]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [13]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [14]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [15]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 50

In [16]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [17]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [18]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [19]:
len(sample_list_filtered)

36

# Hyperparameter tuning for Croston

In [20]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [21]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOBBIES_2_059_CA_1
Progressing: 0.0 %
Currently Running: HOBBIES_1_140_CA_3
Progressing: 2.78 %
Currently Running: HOBBIES_1_417_CA_2
Progressing: 5.56 %
Currently Running: HOBBIES_1_002_CA_1
Progressing: 8.33 %
Currently Running: HOBBIES_2_100_CA_3
Progressing: 11.11 %
Currently Running: HOBBIES_1_047_CA_2
Progressing: 13.89 %
Currently Running: HOBBIES_1_119_CA_4
Progressing: 16.67 %
Currently Running: HOBBIES_1_277_CA_2
Progressing: 19.44 %
Currently Running: HOBBIES_2_053_CA_1
Progressing: 22.22 %
Currently Running: HOBBIES_1_288_CA_3
Progressing: 25.0 %
Currently Running: HOBBIES_1_115_CA_1
Progressing: 27.78 %
Currently Running: HOBBIES_1_145_CA_4
Progressing: 30.56 %
Currently Running: HOBBIES_1_142_CA_1
Progressing: 33.33 %
Currently Running: HOBBIES_1_234_CA_3
Progressing: 36.11 %
Currently Running: HOBBIES_1_255_CA_3
Progressing: 38.89 %
Currently Running: HOBBIES_2_053_CA_2
Progressing: 41.67 %
Currently Running: HOBBIES_1_198_CA_3
Progressing: 44.44 %
Cur

In [22]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOBBIES_2_059_CA_1,5.0,6.449159,0.05
1,HOBBIES_2_059_CA_1,0.0,6.392574,0.05
2,HOBBIES_2_059_CA_1,0.0,6.392574,0.05
3,HOBBIES_2_059_CA_1,32.0,6.392574,0.05
4,HOBBIES_2_059_CA_1,5.0,6.861739,0.05
...,...,...,...,...
5179,HOBBIES_1_008_CA_3,33.0,29.008109,0.90
5180,HOBBIES_1_008_CA_3,93.0,32.600811,0.90
5181,HOBBIES_1_008_CA_3,74.0,86.960081,0.90
5182,HOBBIES_1_008_CA_3,48.0,75.296008,0.90


# Summay all metrics

In [23]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [24]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.02918475336666688 minutes


In [25]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_1,0.05,0.950760,97.7924,104.8517,52.3102,1.222405,1.320112
1,HOBBIES_1_002_CA_1,0.10,0.970013,99.7728,106.3693,54.8233,1.247160,1.330927
2,HOBBIES_1_002_CA_1,0.15,0.987585,101.5801,107.8877,57.2857,1.269752,1.341703
3,HOBBIES_1_002_CA_1,0.20,1.006218,103.4967,109.3217,60.0550,1.293708,1.356020
4,HOBBIES_1_002_CA_1,0.25,1.023114,105.2346,110.7272,62.8283,1.315433,1.368976
...,...,...,...,...,...,...,...,...
643,HOBBIES_2_147_CA_2,0.70,0.868549,86.3154,132.7400,67.9984,2.481568,3.240380
644,HOBBIES_2_147_CA_2,0.75,0.879076,87.3616,133.8188,71.3158,2.511645,3.251460
645,HOBBIES_2_147_CA_2,0.80,0.887703,88.2189,134.5806,74.3062,2.536293,3.267164
646,HOBBIES_2_147_CA_2,0.85,0.893923,88.8370,134.9968,76.8598,2.554064,3.283957


# Check what products has unexpected metrics result

In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [27]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [28]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [32]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [33]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.0 %
Percentage of unexpected values of WMAPE is: 0.0 %
Percentage of unexpected values of SMAPE is: 0.0 %
Percentage of unexpected values of MAPE is: 0.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [34]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [35]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.158981,105.652533,100.885083,128.483186,6.783466,8.072488
0.10,1.039784,91.606456,94.790150,110.248197,6.228868,7.746646
0.15,0.994099,86.267022,91.700786,100.458189,5.964573,7.583643
0.20,0.956411,82.358433,89.086425,93.487142,5.791006,7.458823
0.25,0.928352,79.459794,87.201608,88.789839,5.665768,7.380513
0.30,0.905398,77.180006,85.701064,85.554292,5.576061,7.331365
0.35,0.887955,75.446300,84.651028,83.480108,5.506124,7.304371
0.40,0.879125,74.397653,84.394964,82.763214,5.462187,7.296460
0.45,0.876765,73.945806,84.408142,82.764667,5.436724,7.305851


In [36]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.5
The optimum Alpha based on Mean WMAPE is: 0.5
The optimum Alpha based on Mean SMAPE is: 0.4
The optimum Alpha based on Mean MAPE is: 0.4
The optimum Alpha based on Mean MAE is: 0.5
The optimum Alpha based on Mean RMSE is: 0.4


### Get MEDIAN metrics value of each Alpha

In [37]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,0.918730,81.28970,91.42570,91.63090,4.322392,5.029699
0.10,0.891049,80.37755,92.68075,84.36910,4.092304,4.663543
0.15,0.874044,77.77535,90.30980,81.55500,3.981983,4.598816
0.20,0.834193,77.29500,91.56010,74.80100,3.813912,4.500123
0.25,0.827842,78.16840,91.62160,66.24735,3.710390,4.402866
0.30,0.832269,77.03095,88.27235,65.27980,3.647641,4.329687
0.35,0.846237,76.80580,88.59495,64.28040,3.422083,4.103143
0.40,0.853235,75.24065,88.70255,68.63185,3.280331,4.062788
0.45,0.867336,73.59975,88.56550,73.15770,3.196884,4.034949


In [38]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.25
The optimum Alpha based on Median of WMAPE is: 0.7
The optimum Alpha based on Median of SMAPE is: 0.9
The optimum Alpha based on Median of MAPE is: 0.35
The optimum Alpha based on Median of MAE is: 0.5
The optimum Alpha based on Median of RMSE is: 0.55


# Run Croston model after decide best paramaters

In [39]:
best_alpha = 0.4

In [40]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_008_CA_3,HOBBIES_1_009_CA_2,HOBBIES_1_032_CA_3,HOBBIES_1_047_CA_2,HOBBIES_1_066_CA_4,HOBBIES_1_067_CA_2,HOBBIES_1_076_CA_1,...,HOBBIES_2_116_CA_2,HOBBIES_2_121_CA_2,HOBBIES_2_124_CA_2,HOBBIES_2_126_CA_3,HOBBIES_2_129_CA_1,HOBBIES_2_129_CA_2,HOBBIES_2_132_CA_2,HOBBIES_2_142_CA_2,HOBBIES_2_142_CA_3,HOBBIES_2_147_CA_2
0,11101,1.0,0.0,82.0,7.0,68.0,7.0,1.0,55.0,1.0,...,3.0,0.0,13.0,0.0,2.0,8.0,0.0,32.0,7.0,0.0
1,11102,1.0,0.0,40.0,13.0,30.0,5.0,4.0,5.0,0.0,...,1.0,0.0,17.0,0.0,14.0,21.0,0.0,0.0,22.0,1.0
2,11103,0.0,0.0,23.0,6.0,24.0,5.0,2.0,0.0,2.0,...,3.0,0.0,2.0,0.0,18.0,8.0,0.0,2.0,0.0,2.0
3,11104,1.0,0.0,92.0,3.0,38.0,2.0,2.0,0.0,2.0,...,2.0,0.0,2.0,0.0,12.0,5.0,0.0,2.0,4.0,2.0
4,11105,1.0,0.0,35.0,3.0,50.0,6.0,4.0,2.0,5.0,...,1.0,0.0,1.0,0.0,4.0,6.0,0.0,7.0,27.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,0.0,0.0,74.0,5.0,23.0,2.0,2.0,11.0,6.0,...,0.0,0.0,10.0,0.0,3.0,0.0,0.0,2.0,34.0,4.0
269,11609,3.0,0.0,48.0,9.0,37.0,11.0,1.0,34.0,2.0,...,0.0,0.0,8.0,0.0,1.0,5.0,0.0,15.0,24.0,10.0
270,11610,2.0,0.0,53.0,4.0,34.0,2.0,8.0,3.0,3.0,...,3.0,0.0,22.0,0.0,6.0,2.0,0.0,14.0,21.0,4.0
271,11611,1.0,0.0,71.0,8.0,36.0,6.0,2.0,32.0,0.0,...,3.0,0.0,11.0,0.0,11.0,11.0,0.0,16.0,4.0,2.0


In [41]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [42]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [43]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOBBIES_1_002_CA_1,1.0,1.245308
1,HOBBIES_1_002_CA_1,0.0,1.177110
2,HOBBIES_1_003_CA_1,0.0,0.000000
3,HOBBIES_1_003_CA_1,0.0,0.000000
4,HOBBIES_1_076_CA_1,0.0,3.522465
...,...,...,...
193,HOBBIES_1_202_CA_4,5.0,3.518119
194,HOBBIES_1_223_CA_4,0.0,1.829161
195,HOBBIES_1_223_CA_4,0.0,1.829161
196,HOBBIES_1_277_CA_4,2.0,14.279628


In [44]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [45]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_1,0.711209,142.2417,110.9254,24.5308,0.711209,0.850224
1,HOBBIES_1_003_CA_1,NaN,NaN,NaN,NaN,0.000000,0.000000
2,HOBBIES_1_008_CA_3,0.567125,49.2654,49.3256,71.4863,24.386358,26.339854
3,HOBBIES_1_009_CA_2,2.153411,25.3342,29.0470,25.3620,2.153411,2.155154
4,HOBBIES_1_032_CA_3,0.638886,23.0418,23.2662,23.3558,7.027747,7.068031
...,...,...,...,...,...,...,...
94,HOBBIES_2_129_CA_2,2.541141,50.8228,68.1894,49.2874,5.082282,5.472082
95,HOBBIES_2_132_CA_2,NaN,NaN,NaN,NaN,0.000000,0.000000
96,HOBBIES_2_142_CA_2,1.136566,40.7998,50.0019,39.8122,7.955960,8.329488
97,HOBBIES_2_142_CA_3,0.522848,83.1194,103.4983,113.6008,16.208293,19.113683


In [46]:
df_result_final.isnull().sum()

Product     0
MASE       20
WMAPE      20
SMAPE      20
MAPE       25
MAE         0
RMSE        0
dtype: int64

In [47]:
df_result_final.MAPE.mean()

76.32637297297299

In [48]:
df_result_final.to_csv('Croston_Erratic_Metrics.csv')